In [10]:
import pandas as pd
import numpy as np
from numpy import linalg
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import pathlib

In [2]:
inputFile = '~/data/parsedDaVinciLog.h5'
df_events = pd.read_hdf(inputFile, key='LHCbMC_Lb')

In [52]:
def Similarity(matrix, vector):
    sim = np.dot(
        np.transpose(vector),
        np.dot(
            np.array(matrix),
            np.array(vector)
        )
    )
    
    return sim

In [50]:
def stop_iter(dx, ci, it):
    d1 = linalg.norm(dx)
    d2 = Similarity(ci, dx)
    
    if d2 < 0:
        d2 = 0
        
    ok1 = d1 < 1e-3
    ok2 = d2 < 1e-2
    
    if ok1 or ok2:
        return True
    
    return False

In [61]:
# PS = previous step
# PI = previous iteration
# _ = current value (to return, if necessary)
def step(
    entryPI,
    xPS,
    ciPS,
    chi2PS,
):
    ciPI, xPI, qPI, pmcPI = entryPI
    
    _ci = ciPS + ciPI
    _c = linalg.inv(_ci)
    _x = np.dot(
        _c,
        np.dot(ciPS, xPS) + np.dot(ciPI, xPI)
    )
    
    _dx = xPI - _x
    
    _q = qPI - np.dot(
        pmcPI,
        np.dot(ciPI, _dx)
    )
    
    _dchi2 = Similarity(ciPI, _dx) + Similarity(ciPS, _x - xPS)
    _chi2 = chi2PS + _dchi2
    
    _entry = _ci, _x, _q, _pmc
    return _entry

Simulate the first iteration.

In [70]:
event = df_events.iloc[3]

In [71]:
for dx, ci, it in zip(
    event['iter_deltaVertices'],
    event['iter_covMatrices'],
    range(1,event['numberOfIterations']+1)
):
    print(stop_iter(dx, ci, it))

False
False
False
False
False
True


In [100]:
event['p_posMomCovMatrix']

[[0.00073441, 0.000670873, 0.0, -1.35959, 0.0292281, 29.5204, 29.5376],
 [0.000670873, 0.148071, 0.0, -1.48942, -3.93995, 32.4426, 32.4547],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [-1.35959, -1.48942, 0.0, 19348.4, -643.155, -429949.0, -430179.0],
 [0.0292281, -3.93995, 0.0, -643.155, 204.604, 14320.7, 14328.6],
 [29.5204, 32.4426, 0.0, -429949.0, 14320.7, 9557760.0, 9562880.0],
 [29.5376, 32.4547, 0.0, -430179.0, 14328.6, 9562880.0, 9568010.0]]

In [99]:
np.array(event['p_posMomCovMatrix'])[3:,:3]

array([[-1.35959e+00, -1.48942e+00,  0.00000e+00],
       [ 2.92281e-02, -3.93995e+00,  0.00000e+00],
       [ 2.95204e+01,  3.24426e+01,  0.00000e+00],
       [ 2.95376e+01,  3.24547e+01,  0.00000e+00]])

In [82]:
np.array(event['seed_ci'])*2

array([[ 5.233480e-01, -1.718612e-03, -1.870152e-02],
       [-1.718612e-03,  2.739360e-03, -1.708356e-06],
       [-1.870152e-02, -1.708356e-06,  4.295180e-03]])

In [85]:
event['iter_p_refPoint'][0]

(-38.8914, 8.09641, 5197.18)

In [94]:
np.array(list(event['iter_p_momentum'][0]) + [event['iter_p_energy'][0]])

array([ -808.953 ,    51.0076, 30397.8   , 30423.1   ])

In [111]:
linalg.inv(np.array(event['iter_p_posMomCovMatrix'][0])[:3,:3])

LinAlgError: Singular matrix

In [104]:
linalg.inv(np.array(event['iter_p_posMomCovMatrix'][0]))

LinAlgError: Singular matrix

In [101]:
ciPS = np.array(event['seed_ci'])
xPS = np.array(event['seed_vtx'])
chi2PS = event['seed_chi2']
protonPI = (
    linalg.inv(np.array(event['iter_p_posMomCovMatrix'][0]))[:3,:3], #ciPI
    np.array(event['iter_p_refPoint'][0]), #xPI
    np.array(list(event['iter_p_momentum'][0]) + [event['iter_p_energy'][0]]), #qPI
    np.array(event['iter_p_posMomCovMatrix'][0])[3:,:3], #pmcPI
)

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [69]:
event['seed_chi2']

0.0

False
False
False
False
False
True


In [17]:
dx = df_events.iloc[1]['iter_deltaVertices'][1]
dx

(-34.4484, -25.3509, -502.546)

In [22]:
ci = df_events.iloc[1]['iter_covMatrices'][1]
ci

[[0.0109352, -0.00387316, -0.000279066],
 [-0.00387316, 0.0543013, -0.00330196],
 [-0.000279066, -0.00330196, 0.000301503]]

In [9]:
np.linalg.norm(a)

504.3628084121291

In [13]:
ci = [[1,2,3],[4,5]]
ci

[[1, 2], [3, 4]]

In [28]:
np.dot(
    np.transpose(dx),
    np.dot(np.array(ci), np.array(dx)))

23.458692221953637

In [35]:
df_events.iloc[1]['iter_deltaChi2s']

[823.108, 23.4586, 0.611948, 0.00275324]

In [25]:
np.transpose(dx)

array([ -34.4484,  -25.3509, -502.546 ])